In [1]:
import cupy as cp
import numpy as np
from time import perf_counter
import time
from pyqcu import define
from pyqcu import io
from pyqcu import qcu
import cupyx.scipy.sparse.linalg as csla
import scipy.linalg as sla

print('My rank is ', define.rank)


    @@@@@@######QCU NOTES START######@@@@@@@
    1. The libqcu.so was compiled when pyqcu setup in download_path/PyQCU/lib, please add this path to your LD_LIBRARY_PATH.
    2. The QCU(PyQCU) splite grid by x->y->z->t, lattice by x->y->z->t->p->d->c->c or x->y->z->t->c->s(->p) and x->y->z->t->c->s->c->s(->p).
    3. The QUDA(PyQUDA) splite grid by t->z->y->x, lattice by c->c->x->y->z->t->p->d or c->s->x->y->z->t(->p) and c->s->c->s->x->y->z->t(->p).
    4. The QCU input params in numpy array(dtype=np.int32), argv in  numpy array(dtype=np.float32 or float64) array, set_ptrs in numpy array(dtype=np.int64), other in cupy array(dtype=cp.complex64 or complex128).
    5. The smallest lattice size is (x=4,y=4,z=4,t=8) that QCU support.
    @@@@@@######QCU NOTES END######@@@@@@@
    
My rank is  0


In [2]:
# params = np.array([0]*define._PARAMS_SIZE_, dtype=np.int32)
# params[define._LAT_X_] = 32
# params[define._LAT_Y_] = 32
# params[define._LAT_Z_] = 32
# params[define._LAT_T_] = 32
# params[define._LAT_XYZT_] = 1048576
# params[define._GRID_X_] = 1
# params[define._GRID_Y_] = 1
# params[define._GRID_Z_] = 1
# params[define._GRID_T_] = 1
# params[define._PARITY_] = 0
# params[define._NODE_RANK_] = 0
# params[define._NODE_SIZE_] = 1
# params[define._DAGGER_] = 0
# params[define._MAX_ITER_] = 1e4
# params[define._DATA_TYPE_] = 0
# params[define._SET_INDEX_] = 2
# params[define._SET_PLAN_] = 0
# argv = np.array([0.0]*define._ARGV_SIZE_, dtype=np.float32)
# argv[define._MASS_] = 0.0
# argv[define._TOL_] = 1e-9
# print("Parameters:", params)
# print("Arguments:", argv)

In [3]:
# gauge_filename = f"quda_wilson-dslash-gauge_-{params[define._LAT_X_]}-{params[define._LAT_Y_]}-{params  [define._LAT_Z_]}-{params[define._LAT_T_]}-{params[define._LAT_XYZT_]}-{params[define._GRID_X_]}-{params[define._GRID_Y_]}-{params[define._GRID_Z_]}-{params[define._GRID_T_]}-{params[define._PARITY_]}-{params[define._NODE_RANK_]}-{params[define._NODE_SIZE_]}-{params[define._DAGGER_]}-f.bin"
# print("Gauge filename:", gauge_filename)
# gauge = cp.fromfile(gauge_filename, dtype=cp.complex64,
#                     count=params[define._LAT_XYZT_]*define._LAT_DCC_)
# gauge = io.gauge2ccdptzyx(gauge, params)
# print("Gauge:", gauge)
# print("Gauge data:", gauge.data)
# print("Gauge shape:", gauge.shape)

In [4]:
# set_ptrs = np.array(params, dtype=np.int64)
# print("Set pointers:", set_ptrs)
# print("Set pointers data:", set_ptrs.data)
# qcu.applyInitQcu(set_ptrs, params, argv)

In [5]:
# # give x, b, r, r_tilde, p, v, s, t
# lat_t = params[define._LAT_T_]
# lat_z = params[define._LAT_Z_]
# lat_y = params[define._LAT_Y_]
# lat_x = int(params[define._LAT_X_]/define._LAT_P_)
# lat_d = define._LAT_D_
# lat_s = define._LAT_S_
# lat_p = define._LAT_P_
# lat_c = define._LAT_C_
# latt_shape = (lat_s, lat_c, lat_t, lat_z, lat_y, lat_x)
# MAX_ITER = params[define._MAX_ITER_]
# TOL = argv[define._TOL_]
# N = params[define._LAT_XYZT_] * define._LAT_HALF_SC_
# define._LAT_E_ = 1
# min_eigen_value = 0
# max_eigen_value = 1.6
# degree = 5

In [6]:
# def matvec(src):
#     print("norm src", cp.linalg.norm(src))
#     dest = cp.zeros(N, cp.complex64)
#     qcu.applyWilsonCgDslashQcu(dest, src, gauge, set_ptrs, params)
#     print("norm dest", cp.linalg.norm(dest))
#     return dest


# def _matvec(src, min_eigen_value=min_eigen_value, max_eigen_value=max_eigen_value, matvec=matvec):
#     c = (max_eigen_value + min_eigen_value) / 2
#     e = (max_eigen_value - min_eigen_value) / 2
#     return (matvec(src) - c * src) / e


# def chebyshev_matvec(src, degree=degree, matvec=_matvec):
#     print("norm src", cp.linalg.norm(src))
#     if degree == 0:
#         return src
#     t_prev = src  # T_{k-2}
#     t_curr = matvec(src)  # T_{k-1}
#     t_next = None  # T_k
#     for k in range(2, degree + 1):
#         t_next = 2 * matvec(t_curr) - t_prev
#         t_prev = t_curr
#         t_curr = t_next
#     print("norm dest", cp.linalg.norm(t_next))
#     return t_next

In [7]:
# def power_iteration(matvec, v0, num_iter=params[define._MAX_ITER_], tol=argv[define._TOL_]):
#     v = v0
#     for i in range(num_iter):
#         Av = matvec(v)
#         v_new = Av / cp.linalg.norm(Av)
#         diff = cp.linalg.norm(v_new - v)
#         print(f"Iteration {i+1}: difference = {diff:.6e}")
#         if diff < tol:
#             print(
#                 f"Power Iteration converged after {i+1} iterations. Difference: {diff:.6e}")
#             break
#         v = v_new
#     eigenvalue = cp.dot(v.conj().T, matvec(v))
#     return eigenvalue, v
# def validate_power_iteration(matvec=matvec):
#     v0 = cp.random.randn(N).astype(cp.complex64)
#     eigenvalue, eigenvector = power_iteration(matvec, v0)
#     print("Dominant eigenvalue:", eigenvalue)
#     print("Corresponding eigenvector:", eigenvector[:10])
#     Ax = matvec(eigenvector)
#     eigenvalue_check = cp.dot(eigenvector.conj().T, Ax)
#     print(f"Eigenvalue verification (v^T A v): {eigenvalue_check}")
#     if cp.isclose(eigenvalue, eigenvalue_check):
#         print("Validation passed: Eigenvalue calculation is correct.")
#     else:
#         print("Validation failed: Eigenvalue calculation is incorrect.")
#     print("Eigenvector norm:", cp.linalg.norm(eigenvector))
#     return eigenvalue, eigenvector
# eigenvalue, eigenvector = validate_power_iteration()

In [8]:
# print("Eigenvalue:", eigenvalue)

In [9]:
# # bistabcg
# def dot(a, b):
#     cp.cuda.runtime.deviceSynchronize()
#     return cp.inner(a.flatten().conjugate(), b.flatten())


# def diff(a, b):
#     cp.cuda.runtime.deviceSynchronize()
#     return cp.linalg.norm(a - b) / cp.linalg.norm(b)


# def run(eigen_value):
#     t0 = perf_counter()
#     x = cp.random.rand(N).astype(cp.complex64)
#     print("shape:", x.shape)
#     b = cp.zeros(N, cp.complex64)  # must be zero
#     r = cp.zeros(N, cp.complex64)
#     r_tilde = cp.zeros(N, cp.complex64)
#     p = cp.zeros(N, cp.complex64)
#     s = cp.zeros(N, cp.complex64)
#     v = cp.zeros(N, cp.complex64)
#     t = cp.zeros(N, cp.complex64)
#     r = b - _matvec(x, eigen_value)
#     r_tilde = r
#     r_norm2 = 0
#     rho_prev = 1
#     rho = 0
#     alpha = 1
#     omega = 1
#     beta = 0
#     for i in range(MAX_ITER):
#         # print("## rho:", rho)
#         rho = dot(r_tilde, r)
#         # print("## beta:", beta)
#         beta = (rho / rho_prev) * (alpha / omega)
#         p = r + (p - v * omega) * beta
#         # v = A * p
#         v = _matvec(p, eigen_value)
#         # print("## alpha:", alpha)
#         alpha = rho / dot(r_tilde, v)
#         s = r - v * alpha
#         # t = A * s
#         t = _matvec(s, eigen_value)
#         # print("## omega:", omega)
#         omega = dot(t, s) / dot(t, t)
#         x = x + p * alpha + s * omega
#         r = s - t * omega
#         r_norm2 = cp.linalg.norm(r)
#         print("##{}# r_norm2:{}".format(i, r_norm2))
#         _diff = diff(_matvec(x, 0.0), eigen_value*x)
#         print("diff:", _diff)
#         # break
#         # if (r_norm2 < TOL or i == MAX_ITER - 1):
#         #     print("### turns:", i)
#         #     break
#         if (_diff < TOL or i == MAX_ITER - 1):
#             print("### turns:", i)
#             break
#         rho_prev = rho
#     # print(" diff:", diff(_matvec(x, eigen_value), b))
#     # print("x:", x)
#     t1 = perf_counter()
#     print("Time:", t1-t0)
#     return x, _diff


# eigen_value = eigenvalue
# eigen_vector, _diff = run(eigen_value)
# print("eigen_vector:", eigen_vector)
# print("eigen_vector shape:", eigen_vector.shape)
# print("diff:", _diff)

In [10]:
# print("Difference:", cp.linalg.norm(eigenvector-eigen_vector)/cp.linalg.norm(eigenvector))

In [11]:
# import cupyx.scipy.sparse.linalg as csla
# define._LAT_E_ = 24
# A = csla.LinearOperator((N,
#                         N), matvec=_matvec, dtype=cp.complex64)
# evals, evecs = csla.eigsh(a=A, k=define._LAT_E_, which="SA",
#                           return_eigenvectors=True)
# print("evals:", evals)
# print("evecs:", evecs)
# cp.linalg.norm(_matvec(evecs[:, 0])-evals[0] *
#                evecs[:, 0])/cp.linalg.norm(evals[0]*evecs[:, 0])

In [12]:
# def shifted_power_iteration(
#     matvec,
#     n=N,
#     num_eigenvalues=define._LAT_E_,
#     max_iter=params[define._MAX_ITER_],
#     tol=argv[define._TOL_]
# ):
#     eigenvalues = []
#     eigenvectors = []
#     # Buffer for intermediate results
#     v = cp.ndarray((n,), dtype=np.complex64)
#     w = cp.ndarray((n,), dtype=np.complex64)
#     # Store cumulative shift
#     total_shift = 0.0

#     for k in range(num_eigenvalues):
#         # Initialize random vector
#         v[:] = cp.random.rand(n) + 1j * cp.random.rand(n)
#         # Orthogonalize against previously found eigenvectors
#         for j in range(k):
#             projection = cp.dot(cp.conj(eigenvectors[j]), v)
#             v -= projection * eigenvectors[j]
#         # Normalize
#         v /= cp.linalg.norm(v)

#         # Power iteration
#         eigenvalue = None
#         for iter_count in range(max_iter):
#             # Matrix-vector multiplication
#             w = matvec(v)
#             # Apply shift
#             w += total_shift * v
#             # Calculate Rayleigh quotient
#             new_eigenvalue = cp.dot(cp.conj(v), w)
#             # Normalize
#             norm = cp.linalg.norm(w)
#             if norm < tol:  # Avoid zero vector
#                 v[:] = cp.random.rand(n) + 1j * cp.random.rand(n)
#                 continue
#             w /= norm

#             # Check convergence
#             if eigenvalue is not None:
#                 diff = abs(new_eigenvalue - eigenvalue)
#                 print(f"Iteration {iter_count} for eigenvalue {k}: {diff:.8f}")
#                 if diff < tol:
#                     break
#             eigenvalue = new_eigenvalue
#             v[:] = w

#         # Store results and subtract shift
#         if eigenvalue is not None:
#             eigenvalue -= total_shift
#             eigenvalues.append(eigenvalue)
#             eigenvectors.append(v.copy())
#             # Update shift for next eigenvalue
#             total_shift = eigenvalue.real

#     return eigenvalues, eigenvectors


# # Calculate eigenvalues and eigenvectors
# eigenvalues, eigenvectors = shifted_power_iteration(chebyshev_matvec)

In [13]:
# # Verify results
# print("Computed eigenvalues:")
# for i, ev in enumerate(eigenvalues):
#     print(f"λ_{i} = {ev:.8f}")
#     # Verify eigenvector
#     v = eigenvectors[i]
#     w = cp.zeros_like(v)
#     w = chebyshev_matvec(v)
#     error = cp.linalg.norm(w - ev * v) / cp.linalg.norm(w)
#     print(f"Relative error: {error:.2e}")

In [14]:
def generate_positive_definite_matrix(n):
    """生成随机正定厄米矩阵"""
    A = cp.random.randn(n, n) + 1j * cp.random.randn(n, n)
    return (A @ A.conj().T + n * cp.eye(n))/(n**2)
def chebyshev_filter(A, v, alpha, beta, m):
    """切比雪夫滤波器，针对正定矩阵优化"""
    c = (beta + alpha) / 2
    e = (beta - alpha) / 2
    T = (A - c * cp.eye(A.shape[0])) / e
    v_prev = v
    v_curr = T @ v
    for i in range(1, m):
        v_next = 2 * T @ v_curr - v_prev
        v_prev = v_curr
        v_curr = v_next / cp.linalg.norm(v_next)
    return v_curr
def inverse_iteration_with_chebyshev(A, k, max_iter=1000, tol=1e-10):
    """切比雪夫加速的逆迭代法"""
    n = len(A)
    eigenvalues = []
    eigenvectors = []
    eigs = sla.eigvals(A.get())
    alpha = cp.min(eigs.real)
    beta = cp.max(eigs.real)
    for i in range(k):
        v = cp.random.randn(n) + 1j * cp.random.randn(n)
        v = v / cp.linalg.norm(v)
        if eigenvectors:
            Q = cp.column_stack(eigenvectors)
            v = v - Q @ (Q.conj().T @ v)
            v = v / cp.linalg.norm(v)
        lambda_prev = float('inf')
        for iter in range(max_iter):
            w = chebyshev_filter(A, v, alpha, beta, m=5)
            if eigenvectors:
                Q = cp.column_stack(eigenvectors)
                w = w - Q @ (Q.conj().T @ w)
                w = w / cp.linalg.norm(w)
            lambda_curr = float(cp.real(cp.vdot(w, A @ w)))
            if abs(lambda_curr - lambda_prev) / lambda_curr < tol:
                break
            v = w
            lambda_prev = lambda_curr
            if iter % 10 == 0:
                alpha = max(alpha, lambda_curr * 0.9)
        eigenvalues.append(lambda_curr)
        eigenvectors.append(w)
        beta = alpha * 1.1
    return cp.array(eigenvalues), cp.array(eigenvectors).T
cp.random.seed(42)
n = 1000
k = 24
print("生成正定厄米矩阵...")
A = generate_positive_definite_matrix(n)
print("使用切比雪夫方法求解...")
t0 = time.time()
custom_eigenvalues, custom_eigenvectors = inverse_iteration_with_chebyshev(
    A, k)
custom_time = time.time() - t0
print("计算参考解...")
t0 = time.time()
true_eigenvalues, true_eigenvectors = sla.eigh(A.get())
print("true_eigenvalues: ", true_eigenvalues)
numpy_time = time.time() - t0
custom_idx = cp.argsort(custom_eigenvalues)
custom_eigenvalues = custom_eigenvalues[custom_idx]
custom_eigenvectors = custom_eigenvectors[:, custom_idx]
print(f"\n{'='*50}")
print(f"矩阵大小: {n}x{n}")
print(f"\n计算时间比较:")
print(f"切比雪夫方法: {custom_time:.4f}秒")
print(f"NumPy方法: {numpy_time:.4f}秒")
print(f"\n特征值比较:")
print(f"{'Index':>5} {'True':>15} {'Computed':>15} {'Rel. Error':>12}")
print('-' * 50)
for i in range(k):
    true_val = true_eigenvalues[i]
    comp_val = custom_eigenvalues[i]
    rel_err = abs(true_val - comp_val) / abs(true_val)
    print(f"{i:5d} {true_val:15.8f} {comp_val:15.8f} {rel_err:12.2e}")
print(f"\n特征向量验证 (residual = ||Av - λv||/||v||):")
for i in range(k):
    v = custom_eigenvectors[:, i]
    lambda_i = custom_eigenvalues[i]
    residual = cp.linalg.norm(A @ v - lambda_i * v) / cp.linalg.norm(v)
    print(f"v_{i}: {residual:.2e}")

生成正定厄米矩阵...
使用切比雪夫方法求解...
计算参考解...
true_eigenvalues:  [0.001      0.00100001 0.00100003 0.00100004 0.00100009 0.00100013
 0.00100021 0.00100029 0.00100033 0.00100042 0.00100045 0.00100061
 0.00100075 0.00100083 0.00100102 0.00100117 0.00100142 0.00100153
 0.00100165 0.00100176 0.00100197 0.0010024  0.00100252 0.0010026
 0.00100282 0.00100297 0.00100326 0.00100385 0.00100397 0.00100423
 0.00100482 0.00100491 0.00100532 0.00100569 0.00100615 0.00100632
 0.00100668 0.0010069  0.00100714 0.00100794 0.00100829 0.0010086
 0.00100895 0.00100914 0.00100975 0.00101002 0.00101048 0.00101096
 0.00101131 0.00101169 0.00101257 0.00101301 0.0010132  0.00101398
 0.00101428 0.00101452 0.00101553 0.00101614 0.00101623 0.00101724
 0.00101802 0.00101853 0.0010198  0.00101992 0.00102057 0.00102121
 0.00102198 0.00102285 0.00102342 0.00102393 0.00102506 0.00102538
 0.00102634 0.00102678 0.00102745 0.00102824 0.00102914 0.00102964
 0.00103036 0.00103124 0.00103183 0.00103238 0.00103363 0.00103446
 0.0010351

In [15]:
# qcu.applyEndQcu(set_ptrs, params)